<a href="https://colab.research.google.com/github/Zhen-Miao/PACS/blob/test/vignettes/Notebook_1_Test_For_Sens_Spec_real_kidney_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test PACS sensitivity and specificity


## Step 1. Install packages
Here in this notebook, we show examples to run PACS test on real data. We need to install several packages first. This may take a while.

In [1]:
install.packages('devtools', quiet = TRUE)
install.packages('data.table') ## (please make sure it is newer than 1.8)
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



Installing libgsl for Rfast

In [2]:
system("sudo apt install libgsl-dev")

In [3]:
install.packages('dplyr', quiet = TRUE)
install.packages('Matrix', quiet = TRUE)
install.packages('tidyr', quiet = TRUE)
install.packages('tictoc', quiet = TRUE)
install.packages('Rfast', quiet = TRUE)
install.packages('plyr', quiet = TRUE)
install.packages('logistf', quiet = TRUE)
install.packages('nabor', quiet = TRUE)

Warning message:
“package ‘parallel’ is a base package, and should not be updated”


In [4]:
BiocManager::install(c("GenomeInfoDb","Rsamtools","IRanges","edgeR","SummarizedExperiment"), quiet= TRUE)
devtools::install_github("Zhen-Miao/PIC-snATAC", quiet = TRUE)
devtools::install_github("Zhen-Miao/PACS", quiet = TRUE)
devtools::install_github("immunogenomics/presto", quiet = TRUE)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.17 (BiocManager 1.30.22), R 4.3.1 (2023-06-16)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'GenomeInfoDb' 'Rsamtools' 'IRanges' 'edgeR'
  'SummarizedExperiment'”
Old packages: 'bit', 'bslib', 'curl', 'devtools', 'fontawesome', 'gargle',
  'ggplot2', 'gtable', 'highr', 'httr', 'isoband', 'openssl', 'rmarkdown',
  'roxygen2', 'uuid', 'whisker', 'boot', 'foreign', 'Matrix', 'nlme',
  'survival'



Source scripts from PACS

## Step 2. Load packages

In [11]:
suppressPackageStartupMessages({
  library('SummarizedExperiment', quiet = T)
  library('dplyr', quiet = T)
  library('Matrix', quiet = T)
  library('tidyr', quiet = T)
  library('parallel', quiet = T)
  library('tictoc', quiet = T)
  library('plyr', quiet = T)
  library('logistf', quiet = T)
  library('edgeR', quiet = T)
  library('presto', quiet = T)
  library('nabor', quiet = T)
  library('PICsnATAC', quiet = T)
  library('Rfast', quiet = T)
  })

In [7]:
source("https://raw.githubusercontent.com/Zhen-Miao/PACS/main/R/param_estimate_joint_p_q.R")
source("https://raw.githubusercontent.com/Zhen-Miao/PACS/main/R/CCT_function.R")
source("https://raw.githubusercontent.com/Zhen-Miao/PACS/main/R/differential_identification.R")
source("https://raw.githubusercontent.com/Zhen-Miao/PACS/main/R/param_estimate_simple.R")
source("https://raw.githubusercontent.com/Zhen-Miao/PACS/main/R/param-estimate_logit_get_p_by_t_June.R")
source("https://raw.githubusercontent.com/Zhen-Miao/PACS/main/analysis_codes/other_methods_for_differential_updated.R")
source("https://raw.githubusercontent.com/Zhen-Miao/PACS/main/R/PACS_test_auto.R")
source("https://raw.githubusercontent.com/Zhen-Miao/PACS/main/R/PACS_test_logit.R")
source("https://raw.githubusercontent.com/Zhen-Miao/PACS/main/R/PACS_test_cumulative.R")

## Step 3. Load data.
Here, we have provided a real kidney dataset from Google Drive.

In [8]:
system("gdown --id 1c3A7nunSVV2AYre4jAjoue2i98lI_GHp")
kidney_features_to_keep <- readRDS('kidney_features_to_keep.rds')

In [9]:
system("gdown --id 1GTjP3x_7aVgR2Vo_cMaHwX7Y1NBOfuDH")
load('data_for_test_for_t1e_power.rdata')

In [12]:
system("gdown --id 1JfByDuoSDd0ij-p5w-FLGNQA9cN8n4OJ")
r_by_ct_est = readRDS('r_by_ct_est_kidney_adult.rds')

In this data subset, we have retained two abundant cell types, PT and LOH cells. In each cell type, we retained a random subset of 1000 cells. We construct a peak-by-cell matrix and subset based on the cell types.

In [13]:
pmatpt <- pmats[x.sp_cluster2 == 'PT',kidney_features_to_keep]   ##6308 cells, 159038 features
q_val <- r_by_ct_est$q_vec_new[x.sp_cluster2 == 'PT']
cell_type_labels = x.sp_cluster2[x.sp_cluster2 == 'PT']

In [14]:
actual_pmatpt <- pmats[x.sp_cluster2 %in% c('PT','LOH'),kidney_features_to_keep]
actual_q_val <- r_by_ct_est$q_vec_new[x.sp_cluster2 %in% c('PT','LOH')]
actual_cell_type_labels = x.sp_cluster2[x.sp_cluster2 %in% c('PT','LOH')]

In [15]:
p_by_c <- t(pmatpt)
p_by_c@x[p_by_c@x >= 10] = 0 ## remove the outlier counts
p_by_c = drop0(p_by_c)

actual_p_by_c <- t(actual_pmatpt)
actual_p_by_c@x[actual_p_by_c@x >= 10] = 0 ## remove the outlier counts
actual_p_by_c = drop0(actual_p_by_c)

In [16]:
rs = rowSums(p_by_c)
quantile(rs)

p_by_c = p_by_c[rs > 18,]

actual_rs = rowSums(actual_p_by_c)
quantile(actual_rs)

actual_p_by_c = actual_p_by_c[actual_rs > 18,]

0%   25%   50%   75%  100% 
    0    19    61   224 17382

0%   25%   50%   75%  100% 
    0    33    85   275 20323

Here, we set number of cells to sample as 500. If compute resources allow, you can also set n_cell_sample to 1000.

In [17]:
n_cell_total = ncol(p_by_c)
act_n_cell_total = ncol(actual_p_by_c)
n_repeat <- 5
# n_cell_sample <- 1000
n_cell_sample <- 500
n_features_sample <- 10000 ## here we use all features
n_features_total <- dim(p_by_c)[1]
act_n_features_total <- dim(actual_p_by_c)[1]

In [18]:
set.seed(3384)

We have total of five different methods that we want to test

In [19]:
## store the data in the following matrix
methods_all <- c('our', 'seurat', 'archR', 'snapATAC','fisher')
#############################
p_value_permuted_label <- rep(list(), length = length(methods_all))
names(p_value_permuted_label) <- methods_all
for(m in methods_all){
  p_value_permuted_label[[m]] = matrix(ncol = n_repeat, nrow = n_features_sample)
}
#############################
p_value_actual_label <- rep(list(), length = length(methods_all))
names(p_value_actual_label) <- methods_all
for(m in methods_all){
  p_value_actual_label[[m]] = matrix(ncol = n_repeat, nrow = n_features_sample)
}

In [20]:
p_by_c1 = actual_p_by_c[,actual_cell_type_labels %in% c('PT')]
q_val1 = actual_q_val[actual_cell_type_labels %in% c('PT')]

p_by_c2 = actual_p_by_c[,actual_cell_type_labels %in% c('LOH')]
q_val2 = actual_q_val[actual_cell_type_labels %in% c('LOH')]

cells_sampled1_mat <- matrix(ncol = n_repeat, nrow = n_cell_sample )
cells_sampled2_mat <- matrix(ncol = n_repeat, nrow = n_cell_sample )
act_cells_sampled1_mat <- matrix(ncol = n_repeat, nrow = n_cell_sample )
act_cells_sampled2_mat <- matrix(ncol = n_repeat, nrow = n_cell_sample )
features_sampled_mat <- matrix(ncol = n_repeat, nrow = n_features_sample )

for(iii in 1:5){

  ## only one matrix here
  all_cells1 = sample(1:ncol(p_by_c1), replace = F,size = n_cell_sample)
  all_cells2 = sample(1:ncol(p_by_c2), replace = F,size = n_cell_sample)

  ## random sample some cells
  act_cells_sampled1_mat[,iii] <- all_cells1
  act_cells_sampled2_mat[,iii] <- all_cells2

  # f1 = sample(1:act_n_features_total, replace = F, size = n_features_sample)
  # features_sampled_mat[,iii] <- f1
}

for(iii in 1:5){
  ## only one matrix here
  all_cells = sample(1:(n_cell_total), replace = F,size = n_cell_sample*2)

  ## random sample some cells
  cells_sampled1_mat[,iii] <- all_cells[1:n_cell_sample]
  cells_sampled2_mat[,iii] <- all_cells[(1:n_cell_sample)+n_cell_sample]

  f1 = sample(1:n_features_total, replace = F, size = n_features_sample)
  features_sampled_mat[,iii] <- f1
}

In [ ]:
# cells_sampled1_mat <- readRDS('data/kidney_cells_sampled1_mat.rds')
# cells_sampled2_mat <- readRDS('data/kidney_cells_sampled2_mat.rds')
# features_sampled_mat <- readRDS('data/kidney_features_sampled_mat.rds')
saveRDS(act_cells_sampled1_mat,'data/kidney_cells_sampled1_mat_power.rds')
saveRDS(act_cells_sampled2_mat,'data/kidney_cells_sampled2_mat_power.rds')
saveRDS(cells_sampled1_mat,'data/kidney_cells_sampled1_mat.rds')
saveRDS(cells_sampled2_mat,'data/kidney_cells_sampled2_mat.rds')
saveRDS(features_sampled_mat,'data/kidney_features_sampled_mat.rds')

Warning message in gzfile(file, mode):
“cannot open compressed file 'data/kidney_cells_sampled1_mat_power.rds', probable reason 'No such file or directory'”


ERROR: ignored

## Step 4. P value calculation
Next, we compute the p values for the five pre-determined methods (including PACS) in this part.

In [32]:
#----------- input --------------
for(iii in 1:5){

  ## random sample some cells and peaks
  cells_sampled1 <- cells_sampled1_mat[,iii]
  cells_sampled2 <- cells_sampled2_mat[,iii]
  data_matrix_pos <- p_by_c[,cells_sampled1]
  data_matrix_neg <- p_by_c[,cells_sampled2]
  true_q_pos <- q_val[cells_sampled1]
  true_q_neg <- q_val[cells_sampled2]
##############################
  act_cells_sampled1 <- act_cells_sampled1_mat[,iii]
  act_cells_sampled2 <- act_cells_sampled2_mat[,iii]
  act_data_matrix_pos <- p_by_c1[,act_cells_sampled1]
  act_data_matrix_neg <- p_by_c2[,act_cells_sampled2]
  act_true_q_pos <- q_val1[act_cells_sampled1]
  act_true_q_neg <- q_val2[act_cells_sampled2]


  colnames(data_matrix_pos) <- colnames(data_matrix_neg) <- c(1:n_cell_sample)

  ## get some quantities
  n_p = dim(data_matrix_pos)[2]
  n_n = dim(data_matrix_neg)[2]
  n_reads_cell = c(colSums(data_matrix_pos), colSums(data_matrix_neg))

  ## sample features
  data_matrix_pos = data_matrix_pos[features_sampled_mat[,iii],]
  data_matrix_neg = data_matrix_neg[features_sampled_mat[,iii],]

  ## our method
  group.info <- data.frame(group = c(rep.int(0,times = n_p),
                                     rep.int(1,times = n_n )))

  data_mat = cbind(data_matrix_pos, data_matrix_neg)
  data_mat = Matrix(data_mat,sparse = T)
  rownames(data_mat) = paste('f', 1:(nrow(data_matrix_pos)), sep = '_')

##############################
  colnames(act_data_matrix_pos) <- colnames(act_data_matrix_neg) <- c(1:n_cell_sample)

  ## get some quantities
  act_n_p = dim(act_data_matrix_pos)[2]
  act_n_n = dim(act_data_matrix_neg)[2]
  act_n_reads_cell = c(colSums(act_data_matrix_pos), colSums(act_data_matrix_neg))

  ## sample features
  act_data_matrix_pos = act_data_matrix_pos[features_sampled_mat[,iii],]
  act_data_matrix_neg = act_data_matrix_neg[features_sampled_mat[,iii],]

  ## our method
  act_group.info <- data.frame(group = c(rep.int(0,times = act_n_p),
                                     rep.int(1,times = act_n_n )))

  act_data_mat = cbind(act_data_matrix_pos, act_data_matrix_neg)
  act_data_mat = Matrix(act_data_mat,sparse = T)
  rownames(act_data_mat) = paste('f', 1:(nrow(act_data_matrix_pos)), sep = '_')

  our_p = pacs_test_sparse(
    covariate_meta.data = group.info,
    formula_full = ~ factor(group),
    formula_null = ~ 1,
    pic_matrix = data_mat,
    n_peaks_per_round = NULL, T_proportion_cutoff = 0.2,
    cap_rates = c(true_q_pos, true_q_neg)
  )$pacs_p_val

  act_our_p = pacs_test_sparse(
    covariate_meta.data = act_group.info,
    formula_full = ~ factor(group),
    formula_null = ~ 1,
    pic_matrix = act_data_mat,
    n_peaks_per_round = NULL, T_proportion_cutoff = 0.2,
    cap_rates = c(act_true_q_pos, act_true_q_neg)
  )$pacs_p_val

  p_value_permuted_label[['our']][,iii] = our_p
  p_value_actual_label[['our']][,iii] = act_our_p

  ## seurat
  p_value_permuted_label[['seurat']][,iii] = seurat_method2_subsample(data_matrix_pos, data_matrix_neg,
                           n_reads_cell)
  p_value_actual_label[['seurat']][,iii] = seurat_method2_subsample(act_data_matrix_pos, act_data_matrix_neg,
                           act_n_reads_cell)

  ## snapATAC-- edgeR method
  data_matrix_pos_2= data_matrix_pos
  data_matrix_pos_2@x[data_matrix_pos_2@x != 0] = 1
  data_matrix_neg_2= data_matrix_neg
  data_matrix_neg_2@x[data_matrix_neg_2@x != 0] = 1
  p_value_permuted_label[['snapATAC']][,iii] <-
    snapATAC_method(data_matrix_pos_2, data_matrix_neg_2, bcv = 0.4)
##############################
  act_data_matrix_pos_2= act_data_matrix_pos
  act_data_matrix_pos_2@x[act_data_matrix_pos_2@x != 0] = 1
  act_data_matrix_neg_2= act_data_matrix_neg
  act_data_matrix_neg_2@x[act_data_matrix_neg_2@x != 0] = 1
  p_value_actual_label[['snapATAC']][,iii] <-
    snapATAC_method(act_data_matrix_pos_2, act_data_matrix_neg_2, bcv = 0.4)

  # Fisher's exact test
  p_value_permuted_label[['fisher']][,iii] <-
    fisher_method(data_matrix_pos_2, data_matrix_neg_2)
  p_value_actual_label[['fisher']][,iii] <-
    fisher_method(act_data_matrix_pos_2, act_data_matrix_neg_2)

  ## archR method
  p_value_permuted_label[['archR']][,iii] <- archR_method(data_matrix_pos,data_matrix_neg)
  p_value_actual_label[['archR']][,iii] <- archR_method(act_data_matrix_pos,act_data_matrix_neg)

}

[1] "920 peaks consider cumulative logit models"
[1] "9080 peaks consider logit models"
parameter_estimation: 3.456 sec elapsed
[1] "37 regions fail to converge, try newton"
 f_538  f_813 f_1436 f_1517 f_1600 f_1825 
    59     84    143    154    161    174 
newton method for estimation: 0.129 sec elapsed
[1] "21 regions fail to converge aftere newton"
running parameter_estimation: 2.753 sec elapsed
[1] "37 regions fail to converge, try newton"
 f_538  f_813 f_1436 f_1517 f_1600 f_1825 
    59     84    143    154    161    174 
newton method for estimation: 0.126 sec elapsed
[1] "19 regions fail to converge aftere newton"
computing firth loss start: 0.151 sec elapsed
pacs computing cumulative logit part: 7.175 sec elapsed
parameter_estimation: 16.563 sec elapsed
[1] "0 regions fail to converge, try newton"
named integer(0)
running parameter_estimation: 17.033 sec elapsed
[1] "0 regions fail to converge, try newton"
named integer(0)
computing firth loss start: 0.997 sec elapsed
pacs c

## Step 5. Compute Type 1 error and Power
Now we use the calculated p value for the permuted label to compute type 1 error, and actual labels to compute power of the five different methods. Note that the actual DAR set is unknown, so we set the union of identified differential peaks as the "pseudo-true" set of DAR for power evaluation.

In [33]:
t1power_mat <- matrix(nrow = length(methods_all), ncol = 5)
colnames(t1power_mat) <- c('t1e','t1e_sd','power','power_sd','scenario')
rownames(t1power_mat) <- methods_all
t1power_mat[,'scenario'] <- 1

We will only compute the p values for peaks with at least four cells accessible.

In [34]:
for(jj in methods_all){
  t1e_pic <- colMeans(p_value_permuted_label[[jj]] < 0.05) ## get the type 1 error --> I believe your current approach is exactly this
  t1power_mat[jj,'t1e'] <- mean(t1e_pic)
  t1power_mat[jj,'t1e_sd'] <- sd(t1e_pic)
}


pic_cut <- min(0.05,quantile(p_value_permuted_label[['our']],0.05))
pseu_cut <- min(0.05,quantile(p_value_permuted_label[['seurat']],0.05))
parch_cut <- min(0.05,quantile(p_value_permuted_label[['archR']],0.05))
psnap_cut <- min(0.05,quantile(p_value_permuted_label[['snapATAC']],0.05))
pfisher_cut <- min(0.05,quantile(p_value_permuted_label[['fisher']],0.05))

power_mat = matrix(ncol = 5, nrow = nrow(t1power_mat))
rownames(power_mat) = rownames(t1power_mat)

for(ii in 1:5){
  union_true = p_value_actual_label[['our']][,ii] < pic_cut |
    p_value_actual_label[['seurat']][,ii] < pseu_cut |
    p_value_actual_label[['archR']][,ii] < parch_cut |
    p_value_actual_label[['snapATAC']][,ii] < psnap_cut
  power_mat['our',ii] = sum(p_value_actual_label[['our']][,ii] < pic_cut & union_true) / sum(union_true)
  power_mat['seurat',ii] = sum(p_value_actual_label[['seurat']][,ii] < pseu_cut & union_true) / sum(union_true)
  power_mat['archR',ii] = sum(p_value_actual_label[['archR']][,ii] < parch_cut & union_true) / sum(union_true)
  power_mat['snapATAC',ii] = sum(p_value_actual_label[['snapATAC']][,ii] < psnap_cut & union_true) / sum(union_true)
  power_mat['fisher',ii] = sum(p_value_actual_label[['fisher']][,ii] < pfisher_cut & union_true) / sum(union_true)
}

t1power_mat[,'power'] = rowMeans(power_mat)
t1power_mat[,'power_sd'] = sqrt(Rfast::rowVars(power_mat))

In [35]:
t1power_mat

,t1e,t1e_sd,power,power_sd,scenario
our,0.04514,0.002839542,0.8531904,0.003800042,1
seurat,0.06506,0.002129084,0.8419145,0.012765516,1
archR,0.04438,0.002226432,0.7219423,0.013469915,1
snapATAC,0.00150,0.000543139,0.4274000,0.006539277,1
fisher,0.02640,0.001653784,0.7390755,0.006836519,1


## Processed data
Some processed data can be downloaded here:<br>.
https://drive.google.com/file/d/1eEdziIGLbuSA7OiRpSO8sLy1kg1NJbBM/view?usp=drive_link

https://drive.google.com/file/d/1v2x1HPzVCLXbrow5XeHbZfAqVuhfKnQG/view?usp=drive_link

https://drive.google.com/file/d/1gZgi6g4NBu-XDvcZLAlvy7uUqBj1LKH_/view?usp=drive_link

## Reference
For this worksheet, we used the following public data:
Miao, Z., Balzer, M.S., Ma, Z. et al. Single cell regulatory landscape of the mouse kidney highlights cellular differentiation programs and disease targets. _Nat Commun_ 12, 2277 (2021). https://doi.org/10.1038/s41467-021-22266-1

If you used PIC-snATAC counting in your analysis, please cite our manuscript:

Miao, Z., Wang, J., Park, K., Kuang, D., & Kim, J.. Model-based compound hypothesis testing for snATAC-seq data with PACS. _bioRxiv_, 2023-07.